In [ ]:
import ROOT
import ROOT.ROOT as rr

import uproot
import numpy as np
import pandas as pd
import os
from pathlib import Path
import awkward as ak

import random

import cpp

### Implementation of the skimming step of the analysis

The skimming step reduces the inital generic samples to a dataset optimized for this specific analysis. Most important, the skimming removes all events from the initial dataset, which are not of interest for our study.

In [ ]:
dirBasePath  = '/data/FCNC/'
dirOutPath = '/data/Skim/'
dirPlotPath = './Plots/'

listDir = os.listdir(dirBasePath)

In [ ]:
def returnDir(string):
    return [filename for filename in listDir if filename.startswith(string)]

def loadData(chain, pathDirs):
    # Set up multi-threading capability of ROOT
    rr.EnableImplicitMT()
    
    for Dir in pathDirs:
        print('>>> Process directory ', Dir)
        file_list = os.listdir(dirBasePath + Dir)
        for file in file_list:
            chain.AddFile(dirBasePath + Dir + '/' + file)
            
    return chain

def CountEvents(df):
    n = df.Count().GetValue()
    print('\nNumber of events:', n, '\n')

#### List of all MC background directories

'DYJetsToLL_M', 'QCD', 'ST_s', 'ST_t', 'ST_tW_antitop', 'ST_tW_top', 'TTTT_Tune', 'TTTo2L2Nu', 'TTToHadronic',
'TTToSemiLeptonic', 'TTWJetsToLNu', 'TTZToLLNuNu', 'WGToLNuG', 'WJetsToLNu', 'WWTo2L2Nu', 'WWW', 'WWZ', 'WZG',
'WZTo1L1Nu2Q', 'WZTo2L2Q', 'WZTo3LNu', 'WZZ', 'WmWmJJ', 'WminusH', 'WpWpJJ', 'ZG', 'ZZ', 'tZq'

In [ ]:
# - Data + MC Signal
signalDirs = returnDir('SingleMuon')
signalMCDirs = returnDir('TT_FCNC')

# - MC backgrounds
# listBkgDir = ['DYJetsToLL_M', 'ST_t', 'ST_tW_antitop', 'ST_tW_top', 'TTTo2L2Nu','TTToSemiLeptonic', 'TTWJetsToLNu',
#               'TTZToLLNuNu', 'WGToLNuG', 'WJetsToLNu', 'WZTo3LNu', 'WZZ', 'ZG', 'ZZ', 'tZq']

listBkgDir = ['DYJetsToLL_M', 'QCD', 'ST_s', 'ST_t', 'ST_tW_antitop', 'ST_tW_top', 'TTTT_Tune', 'TTTo2L2Nu', 'TTToHadronic',
              'TTToSemiLeptonic', 'TTWJetsToLNu', 'TTZToLLNuNu', 'WGToLNuG', 'WJetsToLNu', 'WWTo2L2Nu', 'WWW', 'WWZ', 'WZG',
              'WZTo1L1Nu2Q', 'WZTo2L2Q', 'WZTo3LNu', 'WZZ', 'WmWmJJ', 'WminusH', 'WpWpJJ', 'ZG', 'ZZ', 'tZq']

bkgMCDirs = dict(list(zip(listBkgDir, map(returnDir, listBkgDir))))
bkgMCChain = dict(list(zip(listBkgDir, [ROOT.TChain('Events') for _ in range(len(listBkgDir))])))

In [ ]:
# - iSkim1 ########################################################################
def FSkim1(df):
    fdf = df.Filter('iSkim == 1', 'iSkim1')\
            .Filter('nCleanJet >= 4 && nBJet > 0 && nBJet <= 2', 'GoodJet')\
            .Define('maskMu', 'Muon_pfRelIso03_all < 0.15 && Muon_pt > 7')\
            .Define('nGoodMu', 'Sum(maskMu)')\
            .Define('Muon_pt15', 'Muon_pt[maskMu]')\
            .Define('maskEl', 'Electron_pfRelIso03_all < 0.15')\
            .Define('nGoodEl', 'Sum(maskEl)')\
            .Filter('nGoodMu == 2 && nGoodEl == 0', 'GoodEvent')
            
    return fdf

# - iSkim2 ########################################################################
def FSkim2(df):
    fdf = df.Filter('iSkim == 2', 'iSkim2')\
            .Filter('nCleanJet >= 4 && nBJet > 0 && nBJet <= 2', 'GoodJet')\
            .Define('maskMu', 'Muon_pfRelIso03_all < 0.15 && Muon_pt > 27')\
            .Define('nGoodMu', 'Sum(maskMu)')\
            .Define('Muon_pt15', 'Muon_pt[maskMu]')\
            .Define('maskEl', 'Electron_pfRelIso03_all < 0.15 && Electron_pt > 20')\
            .Define('nGoodEl', 'Sum(maskEl)')\
            .Define('Electron_pt15', 'Electron_pt[maskEl]')\
            .Filter('nGoodMu == 1 && nGoodEl == 1', 'GoodEvent')
            
    return fdf

# - iSkim3 ########################################################################
def FSkim3(df):
    fdf = df.Filter('iSkim == 3', 'iSkim3')\
            .Filter('nCleanJet >= 2 && nBJet >= 1', 'GoodJet')\
            .Define('maskMu', 'Muon_pfRelIso03_all < 0.15 && Muon_pt > 15')\
            .Define('nGoodMu', 'Sum(maskMu)')\
            .Define('maskEl', 'Electron_pfRelIso03_all < 0.15')\
            .Define('nGoodEl', 'Sum(maskEl)')\
            .Filter('nGoodMu == 3 && nGoodEl == 0', 'GoodEvent')\
            .Filter('Sum(Muon_charge) != -3 && Sum(Muon_charge) != 3', 'GoodCharge')
    
    return fdf

# - iSkim4 ########################################################################
def FSkim4(df):
    fdf = df.Filter('iSkim == 4', 'iSkim4')\
            .Filter('nCleanJet >= 2 && nBJet >= 1', 'GoodJet')\
            .Define('maskMu', 'Muon_pfRelIso03_all < 0.15 && Muon_pt > 15')\
            .Define('nGoodMu', 'Sum(maskMu)')\
            .Define('Muon_pt15', 'Muon_pt[maskMu]')\
            .Define('maskEl', 'Electron_pfRelIso03_all < 0.15 && Electron_pt > 15')\
            .Define('nGoodEl', 'Sum(maskEl)')\
            .Define('Electron_pt15', 'Electron_pt[maskEl]')\
            .Filter('nGoodMu == 2 && nGoodEl == 1', 'GoodEvent')
            
    return fdf

# - iSkim dictionary ##############################################################

FSkim ={1 : FSkim1, 2 : FSkim2, 3 : FSkim3, 4 : FSkim4}

In [ ]:
def DeclareVariables3(df, title, save=True):
    finalVariables3 = {'mu_pt0','mu_pt1','mu_pt2','mu_eta0','mu_eta1','mu_eta2',
                       'mu_phi0','mu_phi1','mu_phi2','mu_mass0','mu_mass1','mu_mass2',
                       'mu_q0','mu_q1','mu_q2', 'inv_m01', 'inv_m12', 'inv_m02', 'inv_m3',
                       'dR01', 'dR02', 'MET_pt', 'MET_phi', 'eventWeightLumi'}

    define =  FSkim3(df).Define('mu_idx', 'find_idx(Muon_charge, Muon_phi, Muon_eta)')\
                        .Define('mu_pt0', 'Muon_pt[maskMu][mu_idx[0]]')\
                        .Define('mu_pt1', 'Muon_pt[maskMu][mu_idx[1]]')\
                        .Define('mu_pt2', 'Muon_pt[maskMu][mu_idx[2]]')\
                        .Define('mu_eta0', 'Muon_eta[maskMu][mu_idx[0]]')\
                        .Define('mu_eta1', 'Muon_eta[maskMu][mu_idx[1]]')\
                        .Define('mu_eta2', 'Muon_eta[maskMu][mu_idx[2]]')\
                        .Define('mu_phi0', 'Muon_phi[maskMu][mu_idx[0]]')\
                        .Define('mu_phi1', 'Muon_phi[maskMu][mu_idx[1]]')\
                        .Define('mu_phi2', 'Muon_phi[maskMu][mu_idx[2]]')\
                        .Define('mu_mass0', 'Muon_mass[maskMu][mu_idx[0]]')\
                        .Define('mu_mass1', 'Muon_mass[maskMu][mu_idx[1]]')\
                        .Define('mu_mass2', 'Muon_mass[maskMu][mu_idx[2]]')\
                        .Define('mu_q0', 'Muon_charge[maskMu][mu_idx[0]]')\
                        .Define('mu_q1', 'Muon_charge[maskMu][mu_idx[1]]')\
                        .Define('mu_q2', 'Muon_charge[maskMu][mu_idx[2]]')\
                        .Define('inv_m01', 'InvMass2(mu_pt0, mu_pt1, mu_eta0, mu_eta1, mu_phi0, mu_phi1, mu_mass0, mu_mass1)')\
                        .Define('inv_m12', 'InvMass2(mu_pt1, mu_pt2, mu_eta1, mu_eta2, mu_phi1, mu_phi2, mu_mass1, mu_mass2)')\
                        .Define('inv_m02', 'InvMass2(mu_pt0, mu_pt2, mu_eta0, mu_eta2, mu_phi0, mu_phi2, mu_mass0, mu_mass2)')\
                        .Define('inv_m3', 'InvMass3(Muon_pt[maskMu], Muon_eta[maskMu], Muon_phi[maskMu], Muon_mass[maskMu])')\
                        .Filter('abs(inv_m01 - 91.2) > 10 && abs(inv_m02 - 91.2) > 10 && abs(inv_m3 - 91.2) > 10', 'rmZ')\
                        .Define('dR01', 'dR(mu_eta0, mu_eta1, mu_phi0, mu_phi1)')\
                        .Define('dR02', 'dR(mu_eta0, mu_eta2, mu_phi0, mu_phi2)')\
                        .Define('Muon_pt15', 'Muon_pt[maskMu]')
    
    if save:
        define.Snapshot('Events', dirOutPath + title + 'Flat3.root', finalVariables3)
    
    return define

In [ ]:
# - Load Data + MC Signal

chainSig = ROOT.TChain('Events')
dfData = rr.RDataFrame(loadData(chainSig, signalDirs))
# CountEvents(dfData)

chainMC = ROOT.TChain('Events')
dfMCSig = rr.RDataFrame(loadData(chainMC, signalMCDirs))
# CountEvents(dfMCSig)

In [ ]:
# -Load MC backgrounds

dfMCBkg = {}
for key, value in bkgMCDirs.items():
    dfMCBkg[key] = rr.RDataFrame(loadData(bkgMCChain[key], value))
#     CountEvents(dfMCBkg[key])

### Read skimmed file

In [ ]:
%%bash

rm /data/Skim/DataFlat3.root

In [ ]:
%%bash

ls /data/Skim

In [ ]:
DeclareVariables3(dfData, 'Data', save=True)

In [ ]:
file = uproot.open(dirOutPath + 'DataFlat3.root')
tree = file['Events']
# tree.keys()
df = tree.arrays(['mu_q0', 'mu_q1', 'mu_q2'], library='pd')
df

### Histograms: save in `.root` extension

In [ ]:
ROOT.gROOT.SetBatch(True)

################################################################################
# Declare the range of the histogram for each variable
#
# Each entry in the dictionary contains of the variable name as key and a tuple
# specifying the histogram layout as value. The tuple sets the number of bins,
# the lower edge and the upper edge of the histogram.
################################################################################

default_nbins = 30
ranges = {
#         'Electron_pt15': (default_nbins, 15, 200),
        'Muon_pt15': (default_nbins, 15, 200),
        'inv_m3': (default_nbins, 0, 200),
#         'Electron_eta': (default_nbins, -5, 5),
#         'Muon_eta': (default_nbins, -5, 5),
#         'Jet_pt': (default_nbins, 10, 500),
#         'Jet_eta': (default_nbins, -5, 5),
        }

# Book a histogram for a specific variable
def bookHistogram(df, variable, range_):
    return df.Histo1D(rr.RDF.TH1DModel(variable, variable, range_[0], range_[1], range_[2]), variable, 'eventWeightLumi')

# Write a histogram with a given name to the output ROOT file
def writeHistogram(h, name):
    h.SetName(name)
    h.Write()

################################################################################
# Main function of the histogramming step
#
# The function loops over the outputs from the skimming step and produces the
# required histograms for the final plotting.
################################################################################

def main(nSkim):
    # Set up multi-threading capability of ROOT
    rr.EnableImplicitMT()
 
    # Create output file
    tfile = ROOT.TFile(dirPlotPath + 'MuonZqualcosa{}.root'.format(nSkim), 'RECREATE')
    variables = ranges.keys()
    
    fdfData = DeclareVariables3(dfData, '', save=False)
#     fdfMC = FSkim[nSkim](dfMC)
    
    # Loop through skimmed datasets and produce histograms of variables
    hists = {}
    for variable in variables:
        hists[variable] = bookHistogram(fdfData, variable, ranges[variable])

#     hists_mc = {}
#     for variable in variables:
#         hists_mc[variable] = bookHistogram(fdfMC, variable, ranges[variable])

    # Write histograms to output file
    for variable in variables:
        writeHistogram(hists[variable], '{}_{}'.format('data', variable))
#     for variable in variables:
#         writeHistogram(hists_mc[variable], '{}_{}'.format('MCSig', variable))
        
    
    for key, value in dfMCBkg.items():
        fdfMCBkg = DeclareVariables3(value, '', save=False)
        
        hists = {}
        for variable in variables:
            hists[variable] = bookHistogram(fdfMCBkg, variable, ranges[variable])
        for variable in variables:
            writeHistogram(hists[variable], '{}_{}'.format(key, variable))
    
    tfile.Close()

if __name__ == '__main__':
    main(3)

### Final plotting

In [ ]:
################################################################################
# Implementation of the plotting step of the analysis
################################################################################

# Declare a human-readable label for each variable
labels = {
#         'Electron_pt15': 'Electron p_{T} / GeV',
        'Muon_pt15': 'Muon p_{T} / GeV',
        'inv_m3': 'Tri-lepton invariant mass / GeV',
#         'Electron_eta': 'Electron #eta',
#         'Muon_eta': 'Muon #eta',
#         'Jet_pt': 'Jet p_{T} / GeV',
#         'Jet_eta': 'Jet #eta',
        }

# Specify the color for each process

# - Signal
colors = {
        'data': ROOT.TColor.GetColor('#BF2229'),
        'MCSig': ROOT.TColor.GetColor('#00A88F'),
        }
### - MC BKG
random.seed(123)
colorsBkg = {} #ROOT.TColor.GetColor
for i in listBkgDir:
    colorsBkg[i] = ROOT.TColor.GetColor(random.randint(0,255), random.randint(0,255), random.randint(0,255))


# Retrieve a histogram from the input file based on the process and the variable name
def getHistogram(tfile, name, variable, tag=''):
    name = '{}_{}{}'.format(name, variable, tag)
    h = tfile.Get(name)
    if not h:
        raise Exception('Failed to load histogram {}.'.format(name))
    return h

################################################################################

# Main function of the plotting step
def main(variable, nSkim):
    tfile = ROOT.TFile(dirPlotPath + 'MuonZqualcosa{}.root'.format(nSkim), 'READ')

    # Styles
    ROOT.gStyle.SetOptStat(0)

    ROOT.gStyle.SetCanvasBorderMode(0)
    ROOT.gStyle.SetCanvasColor(ROOT.kWhite)
    ROOT.gStyle.SetCanvasDefH(600)
    ROOT.gStyle.SetCanvasDefW(600)
    ROOT.gStyle.SetCanvasDefX(0)
    ROOT.gStyle.SetCanvasDefY(0)

    ROOT.gStyle.SetPadTopMargin(0.08)
    ROOT.gStyle.SetPadBottomMargin(0.13)
    ROOT.gStyle.SetPadLeftMargin(0.16)
    ROOT.gStyle.SetPadRightMargin(0.05)

    ROOT.gStyle.SetHistLineColor(1)
    ROOT.gStyle.SetHistLineStyle(0)
    ROOT.gStyle.SetHistLineWidth(1)
    ROOT.gStyle.SetEndErrorSize(2)
    ROOT.gStyle.SetMarkerStyle(20)

    ROOT.gStyle.SetOptTitle(0)
    ROOT.gStyle.SetTitleFont(42)
    ROOT.gStyle.SetTitleColor(1)
    ROOT.gStyle.SetTitleTextColor(1)
    ROOT.gStyle.SetTitleFillColor(10)
    ROOT.gStyle.SetTitleFontSize(0.05)

    ROOT.gStyle.SetTitleColor(1, 'XYZ')
    ROOT.gStyle.SetTitleFont(42, 'XYZ')
    ROOT.gStyle.SetTitleSize(0.05, 'XYZ')
    ROOT.gStyle.SetTitleXOffset(1.00)
    ROOT.gStyle.SetTitleYOffset(1.60)

    ROOT.gStyle.SetLabelColor(1, 'XYZ')
    ROOT.gStyle.SetLabelFont(42, 'XYZ')
    ROOT.gStyle.SetLabelOffset(0.007, 'XYZ')
    ROOT.gStyle.SetLabelSize(0.04, 'XYZ')

    ROOT.gStyle.SetAxisColor(1, 'XYZ')
    ROOT.gStyle.SetStripDecimals(True)
    ROOT.gStyle.SetTickLength(0.03, 'XYZ')
    ROOT.gStyle.SetNdivisions(510, 'XYZ')
    ROOT.gStyle.SetPadTickX(1)
    ROOT.gStyle.SetPadTickY(1)

    ROOT.gStyle.SetPaperSize(20., 20.)
    ROOT.gStyle.SetHatchesLineWidth(5)
    ROOT.gStyle.SetHatchesSpacing(0.05)

    ROOT.TGaxis.SetExponentOffset(-0.08, 0.01, 'Y')
    
    legend = ROOT.TLegend(0.5, 0.3, 0.9, 0.8)
    legend.SetNColumns(2)

    # Data + MC
    data = getHistogram(tfile, 'data', variable)
#     MCSig = getHistogram(tfile, 'MCSig', variable)
    
    stack = ROOT.THStack('', '')
    for key, value in colorsBkg.items():
        histo = getHistogram(tfile, key, variable)
        histo.SetLineWidth(0)
        histo.SetFillColor(value)
        stack.Add(histo)
        legend.AddEntry(histo, key, 'f')

    # Draw histograms
    data.SetMarkerStyle(20)
    data.SetLineColor(ROOT.kBlack)
    data.SetLineWidth(3)
#     MCSig.SetLineColor(colors['MCSig'])
#     MCSig.SetLineWidth(3)

    c = ROOT.TCanvas('', '', 600, 600)
    
    name = data.GetTitle()
    if name in labels:
        title = labels[name]
    else:
        title = name

#     MCSig.Draw('HIST SAME')
    stack.Draw('hist')
    data.Draw('E1P SAME')
    
    stack.GetXaxis().SetTitle(labels[variable])
    stack.GetYaxis().SetTitle('N_{Events}')
    stack.SetMaximum(max(stack.GetMaximum(), data.GetMaximum()) * 1.2)
    stack.SetMinimum(1.0)

    # Add legend
#     legend.AddEntry(MCSig, 'MCSig', 'f')
    legend.AddEntry(data, 'Data', 'lep')
    legend.SetBorderSize(0)
    legend.Draw()

    # Add title
    latex = ROOT.TLatex()
    latex.SetNDC()
    latex.SetTextSize(0.04)
    latex.SetTextFont(42)
    latex.DrawLatex(0.16, 0.935, '#bf{CMS FCNC}')

    # Save
#     c.SaveAs(dirPlotPath + '{}_{}'.format(variable, nSkim))
    c.SaveAs(dirPlotPath + '{}_{}MuonZqualcosa.png'.format(variable, nSkim))
#     ROOT.gROOT.GetListOfCanvases().Draw()

# Loop over all variable names and make a plot for each
if __name__ == '__main__':
    for variable in labels.keys():
        main(variable, 3)

In [ ]:
from IPython.display import Image
for image in sorted(os.listdir(dirPlotPath)):
    if image.endswith('qualcosa.png'):
        print(image + '\n')
        display(Image(filename=(dirPlotPath + image)))

***
### Examples and trials

### Using UPROOT

In [ ]:
def compute_m(p1, p2, phi1, phi2, eta1, eta2):
    return np.sqrt(2*p1*p2*(np.cosh(eta1-eta2) - np.cos(phi1-phi2)))

In [ ]:
file = uproot.open(dirOutPath + 'DataFlat3.root')
tree = file['Events']
# tree.keys()
df = tree.arrays(library='pd')

In [ ]:
df['m_inv01'] = df.apply(lambda x: compute_m(x.mu_pt1, x.mu_pt0, x.mu_phi1, x.mu_phi0, x.mu_eta1, x.mu_eta0), axis=1)

In [ ]:
df.iloc[100:130]

In [ ]:
z = df[df['mu_q1']*df['mu_q0'] == -1]

In [ ]:
import matplotlib.pyplot as plt

plt.hist(z.m_inv01, bins=100)
plt.hist(df.m_inv01, bins=100, histtype='step')
# plt.vlines(90, 0, 300, color='red')
plt.show()

In [ ]:
files = uproot.concatenate([(samplesBasePath + folder + '/*.root:Events') for folder in signalDirs])
dataframe = ak.to_pandas(files)
dataframe

### Flattening trial

In [ ]:
flat = FSkim3(dfData).Define('Muon_0', 'Muon_pt15[0]')\
                     .Define('Muon_2', 'Muon_pt15[2]')

In [ ]:
prova = flat.AsNumpy(columns=['Muon_0'])
prova

### Histogram example

In [ ]:
histo = FSkim3(dfData).Histo1D(rr.RDF.TH1DModel('nJet', 'nJet', 15, 0, 15), 'nJet')
c = ROOT.TCanvas('Hist','',800,600);
c.cd();
histo.Draw();
c.Draw();

In [ ]:
%%cpp
// example of filtering operation MC events
using namespace ROOT;

RDataFrame d('Events', '/data/FCNC/TT_FCNC-aTtoHJ_Tleptonic_HToWWZZtautau_eta_hct-MadGraph5-pythia8_RunIISummer16/6319226B-F5F4-1B45-950E-36BE5993AA49_Skim.root');
//auto c = d.Filter('iSkim != 1 && iSkim != 2 && iSkim != 3 && iSkim != 4').Count();
//std::cout << *c << std::endl;
                
hz = d.Histo1D('nElectron');

### Processing file by file w/o `Chain`

In [ ]:
# def main():

# Set up multi-threading capability of ROOT
rr.EnableImplicitMT()

for signalDir in signalDirs:
    print('>>> Process directory ', signalDir, ':')

#     df = rr.RDataFrame('Events' samplesBasePath + signalDir + '/*.root')
#     print('Number of events:', df.Count())

    df2 = MinimalSelection(df)
    
    df2.Report().Snapshot('Events', dirOutPath + signalDir + 'Fltr.root', finalVariables);

***
Loading of `.root` files through flattening with `uproot`.

In [ ]:
uproot.lazy([basepath+'SingleMuon_Run2017B-Nano1June2019-v1/*.root:Events'], filter_name='nMuon')

In [ ]:
uproot.concatenate([basepath + '{}/*.root:events'.format(folder for folder in return_dir('SingleMuon'))], filter_name='nMuon')

#### To clarify:
- istogrammi (MC inutili e/o raggruppati)
- criteri per i tagli + iSkim (questione Skim2)
- nJet +- stringente di BJet
- i jets: p_t da ncleanJet
- stiamo considerando processi TT e ST? i paper sono su TT: i processi finali?
- i tau?
- e la massa invariante: problema a 15GeV per iSkim3
- Z_mass

#### To do:
- m_eff
- jets

#### Prossimamente:
- BDT: discriminazione segnale/bkg
- Esame MAPD